Implement dropout algorithms in your neural network.


In [ ]:
import numpy as np

class Dropout:
    def __init__(self, drop_prob=0.5):
        assert 0.0 <= drop_prob < 1.0
        self.drop_prob = drop_prob
        self.mask = None

    def forward(self, X, training=True):
        """Apply dropout during training."""
        if training:
            # Sample mask from Bernoulli distribution
            self.mask = (np.random.rand(*X.shape) > self.drop_prob).astype(float)
            # Scale the output to maintain expected value
            return (X * self.mask) / (1.0 - self.drop_prob)
        else:
            # During inference, dropout is not applied
            return X

    def backward(self, dY):
        """Backpropagate through dropout."""
        return (dY * self.mask) / (1.0 - self.drop_prob)
